In [ ]:
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [ ]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://c8432d35-2370-4de9-924c-af3802cf315d.deepnoteproject.com/7000/


In [ ]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
    <link name="table_top_link">
      <inertial>
        <mass>18.70</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

In [ ]:
table_top_sdf_file

'/tmp/robotlocomotion_drake_mkjliJ/table_top.sdf'

# Adding Models 

In this exercise, you will learn the basics of adding objects into Drake. You will learn about URDFs, SDFs, and obj files. 

Parts of this notebook were taken from the [Authoring a Multibody Simulation tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-add293478aac40a984845aa3705eefdd?). Still, it is useful to take a look at the tutorial to learn more about the concepts.

A high level overview of the different formats around describing simulations in Drake:
- URDF and SDF are XML formats that can specify the kinematic and dynamic properties of robots or objects. SDF is the more scalable format, and it is the format that is recommended. However, they should be mostly interchangeable with small syntax differences.
- .obj on the other hand, is a file format that describes the *geometry* of an object, such as a mesh of the surface of the object.
  - There exist other geometry formats, such as .stl, and .dae. These are not well-supported by Drake directly, but [Meshlab](https://www.meshlab.net/), an open-source software, is a handy tool to convert common formats to a `.obj` for you.
- [Model directives](https://drake.mit.edu/doxygen_cxx/structdrake_1_1multibody_1_1parsing_1_1_model_directive.html) are a YAML specification that makes it easy to load multiple URDF and SDF objects into a simulation. They're used at the bottom of this notebook.


The most important formats for creating multibody scenarios in Drake are the [Unified Robot Description Format (URDF)](http://wiki.ros.org/urdf) and the [Simulation Description Format (SDFormat)](http://sdformat.org/).

They are both XML formats to describe robots or objects for robot simulators or visualization, and are fairly similar in syntax.

In a high-level sense, you express different components of your robot using `<link>` tags and connect them via `<joint>` tags. Each `<link>` has three major subtags, `<visual>`, `<collision>`, and `<inertial>`, for its visualization, planning/collision checking, and dynamics aspects. For `<visual>` and `<collision>`, you can either use primitives (box, sphere, cylinder, etc.) or meshes (.obj) to represent the underlying geometry.

The supported primitive types and the supported file types for meshes are described [here](https://drake.mit.edu/doxygen_cxx/group__geometry__file__formats.html). 

### Written Question
Compare and contrast two SDF files of the same YCB sugar box: [File 1](https://github.com/RobotLocomotion/drake/blob/8ed3a0264c521ffc1ea8861f5701b81dfb696af8/manipulation/models/ycb/sdf/004_sugar_box.sdf) and [File 2](https://github.com/RussTedrake/manipulation/blob/24f122591f72cc78931ea3f461d0c34845dc9aca/manipulation/models/hydro/004_sugar_box.sdf). File 2 uses hydroelastic contact, and file 1 uses the default point contact model.

[Hydroelastic contact](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html) in Drake can be used to create compliant collision geometries. This is a great tool for improving the contact dynamics between simulated objects, compared to traditional rigid contact geometries.

Answer the following questions in your written submission.

**Part (a):**  How is the collision geometry (not contact models!) different in File 1 than File 2? What tags enable hydroelastic contact in File 2?

**Part (b):**  In both files, we use the sugar box obj mesh as the visual geometry. For File 2, we could have used the obj mesh as the collision geometry too. In one sentence, why might we prefer to specify the collision geometry using primitives, rather than using the obj mesh? 

## Adding your own models

Now, you will add an object of your choosing into simulation! You have a lot of freedom here. Ideally, this object can be used in your final project. It doesn't have to be an object your robot can pick up; it can be anything you want in your simulation. 

- The only requirement is it cannot be directly taken from the existing models in Drake without modification.
- Your model file can be in any of the formats supported by the [Parser](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_parser.html), but we recommend using SDF. Your model should use hydroelastic contact.

One option is to find a 3D model online, such as at [TurboSquid](https://www.turbosquid.com/3d-model/free), and convert it into a .obj file. The .obj file can then be directly turned into a SDF file by using the below terminal command, documented [here](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html). On Deepnote, you can open a terminal on the left bar and run the command after you have uploaded your .obj file. 
```sh
python3 -m pydrake.multibody.mesh_to_model path/to/mesh.obj
```
We have a more advanced version of this tool available in `manipulation/create_sdf_from_mesh.py`, which additionally supports convex decomposition of a nonconvex mesh and mesh simplification. This isn't needed for this problem, but may be very useful for your project.

Another option is to purely use SDF and geometric primitives to compose primitives (and maybe meshes) to build up your object. 

Here's some more examples of various SDF files to use as inspiration or a guide: 
- [Rubik's Cube](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/02-Lets-get-you-a-robot-8f86172b-b597-4ceb-9bad-92d11ac7a6cc/notebook/rubiks_cube-35164353b13d4a47910ca14e588c74d6?)
- [Drake examples](https://github.com/RobotLocomotion/drake/tree/master/examples), which includes things such as a bouncing ball. 
- If there was a [final project](https://manipulation.csail.mit.edu/misc.html#projects) that had objects you'd like to use, see if their code is open source or ask!

Once you have created your model file, fill in the **absolute path** below. On Deepnote, there is an option on the left bar to copy paths into your clipboard. 

**Part (c):** In your written submission, briefly describe what your object is. Take a screenshot of your object on the table in simulation (the final code cell). Your screenshot should include red contact force arrows and not green contact force arrows, which indicate that you are using hydroelastic contact. 

In [ ]:
your_model_filename = "/root/work/Board.sdf" # Write the absolute path to your file here
knight = "/root/work/BlackKnight.sdf"


It is very helpful to visualize and check your model prior to putting it into simulation. Run the below cell to do so. 

Under the `Scene > drake` dropdown in MeshCat, you can enable viewing 'illustration', 'inertia', and 'proximity' to see the visual model, inertia matrix, and collision model, respectively. You can also adjust the poses of the links in the sliders. 

In [ ]:
if your_model_filename:
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().AddModels(your_model_filename)
    visualizer.Run(loop_once=not running_as_notebook)

INFO:drake:Click 'Stop Running' or press Esc to quit


In [ ]:
def convert_to_coords(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return "" + str(.125*(y-3.5)) + ", "+ str(.125*(x-3.5)) 

def add_piece(piece_id, name,  pos):
    return f"""
- add_model:
    name: {piece_id}
    file: file:///root/work/{name}.sdf
    default_free_body_pose:
        {name}:
            translation: [{convert_to_coords(pos)}, 5.5]
            rotation: !Rpy {{ deg: [0, 0, 0] }}"""


Now, we'll create the simulation with your model. The simulation is replayable in MeshCat. A YCB sugar box has been added to the simulation as a reference, but you may comment it out and remove it.

You will have to replace the link specified in the model directives based on your file. You may have to adjust the starting pose of your model, physical properties of your model, and/or the size of the table (defined at the top of the file) in order to not have your object fall through the world.

In [ ]:
def create_scene():
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://{table_top_sdf_file}
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
    if your_model_filename:
        scenario_data += f"""
- add_model:
    name: articulated_worm
    file: file://{your_model_filename}
    default_free_body_pose:
        Board:
            translation: [0, 0, 2.5]
            rotation: !Rpy {{ deg: [90, 0, 0] }}    
"""
    scenario_data += add_piece('p1', 'BlackKnight',  'b8')
    scenario_data += add_piece('p2', 'BlackBishop',  'c8')
    scenario_data += add_piece('p3', 'BlackKnight',  'g8')
    scenario_data += add_piece('p4', 'BlackBishop',  'f8')
    scenario_data += add_piece('p5', 'BlackQueen',  'd8')
    scenario_data += add_piece('p6', 'BlackRook',  'a8')
    scenario_data += add_piece('p7', 'BlackRook',  'h8')
    scenario_data += add_piece('p8', 'BlackKing',  'e8') 
    scenario_data += add_piece('p9', 'WhiteKnight',  'b1')
    scenario_data += add_piece('p10', 'WhiteBishop',  'c1')
    scenario_data += add_piece('p11', 'WhiteKnight',  'g1')
    scenario_data += add_piece('p12', 'WhiteBishop',  'f1')
    scenario_data += add_piece('p13', 'WhiteQueen',  'd1')
    scenario_data += add_piece('p14', 'WhiteRook',  'a1')
    scenario_data += add_piece('p15', 'WhiteRook',  'h1')
    scenario_data += add_piece('p16', 'WhiteKing',  'e1')
    #for i in range(8): 
    #    scenario_data += add_piece(f"p{17+i}", 'WhitePawn',  f"{chr(97+i)}2")
    #    scenario_data += add_piece(f"p{25+i}", 'BlackPawn',  f"{chr(97+i)}7")

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


create_scene()

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



## How will this notebook be Graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

This notebook only consists of written questions, so please submit all responses in your PDF submission to Gradescope. For this exercise, you should have completed the following subparts:
- Correct answers to part (a)
- Correct answers to part (b)
- A screenshot and description of your added object for part (c)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8432d35-2370-4de9-924c-af3802cf315d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>